In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_parquet(path='generated_1m_cleaned.parquet')
data.head()

,Unnamed: 0,TIC,mjd,mag,magerr,chi2,cadence_id,cadence_filter,cadence_oid,offset,flare_time
0,0,25081629,"[58658.25988, 58658.26033, 58658.26901, 58658....","[17.44388909, 17.38453758, 17.41288879, 17.409...","[0.01609462, 0.02436915, 0.02209847, 0.0227243...",7.709873,181534,2,279205200027182,58658.298365,58380.795196
1,1,149216532,"[58717.17528, 58717.17573, 58717.18701, 58717....","[17.26973991, 17.32330035, 17.29629534, 16.811...","[0.01710255, 0.01801667, 0.0191845, 0.01532426...",80.424814,159648,2,437211300030064,58717.195174,58325.506902
2,2,300037002,"[58697.24395, 58697.2444, 58697.24631, 58697.2...","[20.31556067, 20.51272001, 20.4058346, 20.6033...","[0.13555452, 0.13270269, 0.14270446, 0.1282112...",4.654884,108026,2,383206200105601,58697.292286,58341.948007
3,3,166739932,"[58289.27643, 58289.27734, 58289.27779, 58289....","[18.17937814, 18.2522643, 18.22685253, 18.2658...","[0.04319355, 0.03298203, 0.03667065, 0.0338194...",51.855485,207792,2,330209400077069,58289.318518,58360.185544
4,4,471015484,"[58320.28815, 58320.28862, 58320.28907, 58320....","[16.86832672, 16.88526705, 16.84090475, 16.806...","[0.0208983, 0.01636073, 0.01798354, 0.0146999,...",19.300331,12894,2,334207300000612,58320.290658,58363.742384


In [3]:
list_TIC = [2761472, 201792346, 141334293, 156002545]
data = data.query('TIC in @list_TIC').reset_index(drop=True)
data['max_mag'] = data['mag'].apply(lambda x: np.max(x))
data['min_mag'] = data['mag'].apply(lambda x: np.min(x))
data['max_ampl'] = data['max_mag'] - data['min_mag']
data['len_mag'] = data['mag'].apply(lambda x: len(x))

In [4]:
data = data.sort_values(by=['len_mag', 'max_ampl'], ascending=False)
data.head()

,Unnamed: 0,TIC,mjd,mag,magerr,chi2,cadence_id,cadence_filter,cadence_oid,offset,flare_time,max_mag,min_mag,max_ampl,len_mag
7144,252297,141334293,"[58468.09272, 58468.09317, 58468.09364, 58468....","[17.04100294, 17.08318987, 17.06402786, 17.037...","[0.01541665, 0.01715585, 0.01791738, 0.0154814...",1060.540801,33851,2,803201400020314,58468.174817,58350.396518,17.118483,15.255284,1.863199,345
11414,405115,201792346,"[58468.09273, 58468.09319, 58468.09365, 58468....","[18.94514425, 18.96498083, 18.9962866, 18.9931...","[0.05836118, 0.05768667, 0.05260395, 0.0713114...",13.267403,34963,2,803206100002335,58468.174796,58365.519127,19.191607,18.179429,1.012178,345
9963,352051,201792346,"[58468.09261, 58468.09306, 58468.09353, 58468....","[17.62790494, 17.60118237, 17.64092474, 17.604...","[0.01939597, 0.02026203, 0.02472954, 0.0198014...",60.458458,34998,2,803211400038585,58468.174676,58365.519127,17.691955,16.808120,0.883835,345
2544,89744,141334293,"[58468.09234, 58468.09279, 58468.09326, 58468....","[19.61281192, 19.62987486, 19.59688561, 19.594...","[0.08534756, 0.09627757, 0.0739489, 0.08756327...",102.326371,35492,2,803212300022729,58468.174437,58350.396518,19.912920,17.801454,2.111466,343
9087,320281,201792346,"[58468.09249, 58468.09294, 58468.0934, 58468.0...","[19.9211849, 19.84197264, 19.85458008, 19.7114...","[0.08576737, 0.09811688, 0.12523413, 0.0964323...",61.661098,35375,2,803216200062695,58468.174900,58375.081986,20.071431,18.261618,1.809813,343


In [5]:
def plot_n_random_samples_interpolate_vs_data(
        path_to_file,
        n_grid,
        interpolator_names,
        n_samples,
        positive_selection
    ):
    """Plot n random samples from initial dataframe VS interpolated data"""
    import random

    data = pd.read_parquet(path_to_file)
    if positive_selection:
        twice_mean_chi = 2 * np.mean(data['chi2'])
        data = data[((data['chi2'] > twice_mean_chi) & (len(data['mjd']) > 50))]
    else:
        twice_mean_chi = 2 * np.mean(data['reduced_chi_square'])
        data = data[((data['reduced_chi_square'] > twice_mean_chi) & (len(data['mjd']) > 50))]

    data = data.reset_index()
    indexs = random.sample(range(data.shape[0]), n_samples)
    
    plot_samples_interpolate_vs_data(data, indexs, interpolator_names, n_grid, positive_selection)

def plot_samples_interpolate_vs_data(data, indexs, interpolator_names, n_grid, positive_selection):
    for i in indexs:
        for y_label in ['mag']:
            for interpolator_name in interpolator_names:
                x = data.loc[i, 'mjd']
                y = data.loc[i, y_label]
                interpolator = fit_interpolator(interpolator_name, x, y)
                xnew = np.linspace(x.min(), x.max(), n_grid)
                ynew = interpolator(xnew)
                lst = [[x, y, 'real'],
                        [xnew, ynew, 'iterpolate']]
                df = pd.DataFrame(lst, columns=['x', 'y', 'label'])
                title=''
                if positive_selection:
                    title = (
                        f"Variable {y_label} TIC {data.loc[i, 'TIC']}"
                        f" cadence_id {data.loc[i, 'cadence_id']}"
                        f" interpolator {interpolator_name}"
                    ) 
                else:
                    title = (
                        f"Variable {y_label} oid {data.loc[i, 'oid']}"
                        f" mjd[0] {data.loc[i, 'mjd'][0]}"
                        f" interpolator {interpolator_name}"
                    ) 
                plot_real_VS_interpolate(
                    df,
                    y_label=y_label,
                    title=title
                )



def fit_interpolator(interpolator, x, y):
    """Fit interpolator for initial data"""
    from scipy.interpolate import CubicSpline, UnivariateSpline, Akima1DInterpolator

    interpolator_object = None
    if interpolator == "CubicSpline":
        interpolator_object = CubicSpline(x, y)
    elif interpolator == "Akima1DInterpolator":
        interpolator_object = Akima1DInterpolator(x, y)
    elif interpolator == "UnivariateSpline":
        interpolator_object = UnivariateSpline(x, y, s=0.01)
    else:
        raise ValueError("Undefined interpolator name")
    return interpolator_object


def plot_real_VS_interpolate(df, y_label, title):
    """Plot data via plotly library"""
    import plotly.graph_objects as go
    import os
    if not os.path.exists("images"):
        os.mkdir("images")

    fig = go.Figure()
    for _, row in df.iterrows():
        fig.add_trace(
            go.Scatter(
                x=row['x'],
                y=row['y'],
                mode='lines+markers',
                name=row['label']
            )
        )

    fig.update_layout(
        title=title,
        xaxis_title='MJD',
        yaxis_title=y_label)
    fig.write_html(f'images/{title}.html')
    fig.show()

In [6]:
interpolator_names=['UnivariateSpline','Akima1DInterpolator','CubicSpline']
# interpolator_names=['Akima1DInterpolator']
plot_samples_interpolate_vs_data(
    data,
    range(5),
    interpolator_names,
    n_grid=100,
    positive_selection=True
)

In [8]:
plot_n_random_samples_interpolate_vs_data(
    path_to_file='generated_1m_cleaned.parquet',
    n_grid=100,
    interpolator_names=['UnivariateSpline','Akima1DInterpolator','CubicSpline'],
    positive_selection=True,
    n_samples=5
)

In [9]:
plot_n_random_samples_interpolate_vs_data(
    path_to_file='lc_1M.parquet',
    n_grid=100,
    interpolator_names=['UnivariateSpline','Akima1DInterpolator','CubicSpline'],
    positive_selection=False,
    n_samples=5
)